In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yahooquery as yq

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [75]:
emp = pd.read_csv('D:\what\ibovpred\IBOVDia_10-07-23.csv', sep=';', encoding='latin-1')
emp = emp['Código'] + '.SA'
emp = emp.iloc[0:86]

In [94]:
# Fetch historical data for each stock
stock_data = []
stock_dataAux = []

stock_dataAux = pd.DataFrame(stock_dataAux)

for symbol in emp:
    stock_data = yq.Ticker(symbol).history(period='2y', interval='1d')['close']
    stock_data = pd.DataFrame(stock_data)
    stock_data.reset_index(inplace=True)
    stock_dataAux = pd.concat([stock_dataAux, stock_data], axis=1)
    
stock_dataAux = stock_dataAux['close']
stock_dataAux.columns = emp
stock_dataAux

Código,RRRP3.SA,ALSO3.SA,ALPA4.SA,ABEV3.SA,ARZZ3.SA,ASAI3.SA,AZUL4.SA,B3SA3.SA,BBSE3.SA,BBDC3.SA,...,VIVT3.SA,TIMS3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VIIA3.SA,VBBR3.SA,WEGE3.SA,YDUQ3.SA
0,46.529999,28.430000,52.860001,17.660000,96.620003,17.309999,42.549999,16.709999,23.170000,19.272726,...,42.369999,11.54,37.740002,19.389999,20.230000,115.070000,14.73,29.389999,35.580002,31.740000
1,46.220001,29.000000,52.630001,17.799999,97.129997,17.610001,42.490002,16.950001,22.799999,19.336363,...,41.799999,11.77,38.090000,19.400000,20.500000,115.750000,14.86,29.209999,35.180000,32.369999
2,45.400002,29.500000,52.490002,17.910000,99.459999,17.542000,42.549999,17.120001,23.100000,19.581818,...,41.720001,11.86,37.880001,19.670000,19.790001,115.120003,14.86,29.350000,35.009998,32.759998
3,43.549999,29.180000,52.220001,17.620001,100.809998,17.256001,41.849998,17.000000,22.709999,19.490910,...,41.250000,11.87,38.480000,19.280001,19.760000,115.480003,14.73,29.000000,34.919998,32.330002
4,43.389999,28.780001,52.180000,17.430000,99.570000,17.488001,40.500000,16.590000,22.500000,19.154545,...,41.209999,11.85,37.880001,19.010000,19.330000,113.400002,14.60,28.520000,34.290001,31.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,30.340000,24.959999,9.330000,15.140000,81.300003,13.610000,21.129999,14.740000,31.629999,14.670000,...,42.849998,14.47,30.290001,18.719999,7.320000,65.900002,2.17,17.799999,37.189999,20.150000
495,30.950001,25.370001,9.420000,15.210000,85.000000,13.720000,20.469999,14.730000,31.510000,14.690000,...,42.950001,14.37,30.830000,18.850000,7.380000,65.320000,2.22,18.230000,37.500000,20.620001
496,30.129999,25.040001,9.120000,14.830000,83.650002,13.590000,19.139999,14.180000,31.260000,14.350000,...,42.400002,14.08,29.950001,18.450001,7.370000,64.849998,2.07,17.910000,36.889999,19.950001
497,32.250000,25.469999,9.200000,14.850000,83.550003,14.000000,19.320000,14.360000,31.320000,14.500000,...,42.110001,14.19,30.500000,19.299999,7.530000,65.459999,2.12,18.309999,37.000000,20.000000


In [163]:
#IBOV data
ibov = yq.Ticker('^BVSP').history(period='2y', interval='1d')['close']
ibov = pd.DataFrame(ibov)
ibov.reset_index(inplace=True)
data = pd.concat([ibov, stock_dataAux], axis=1)

data = data.iloc[:, 1:87]
#Renaming the close column to ibov

data.rename(columns={'close': 'ibov'}, inplace=True)

data.shape

(499, 86)

In [158]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
data_treinamento = data.iloc[:, 2:86].values

normalizador = MinMaxScaler(feature_range=(0,1))
data_treinamento_normalizada = normalizador.fit_transform(data_treinamento)

normalizador_previsao = MinMaxScaler(feature_range=(0,1))
normalizador_previsao.fit_transform(data_treinamento[:,1:2])

array([[0.92313221],
       [0.96408046],
       [1.        ],
       [0.97701152],
       [0.94827591],
       [0.92887933],
       [0.9719828 ],
       [0.95761493],
       [0.96839077],
       [0.93821835],
       [0.92959773],
       [0.90660925],
       [0.94252879],
       [0.99353447],
       [0.91091956],
       [0.92672411],
       [0.90660925],
       [0.88577585],
       [0.85057472],
       [0.86853449],
       [0.88936775],
       [0.8448276 ],
       [0.8448276 ],
       [0.78879307],
       [0.79310351],
       [0.75502875],
       [0.68390808],
       [0.6688218 ],
       [0.73204026],
       [0.73419535],
       [0.69181028],
       [0.74353451],
       [0.73563216],
       [0.67744255],
       [0.71623558],
       [0.68031604],
       [0.68390808],
       [0.6688218 ],
       [0.59913794],
       [0.60560347],
       [0.63433907],
       [0.5596265 ],
       [0.58189658],
       [0.58117817],
       [0.63002877],
       [0.60488506],
       [0.57686786],
       [0.565

In [161]:
previsores = []
preco_real = []

for i in range(15, 316):
    previsores.append(data_treinamento_normalizada[i-15:i, :])
    preco_real.append(data_treinamento_normalizada[i, 0])
previsores, preco_real = np.array(previsores), np.array(preco_real)

# Validate and reshape the input shape of previsores
num_samples, timesteps, num_features = previsores.shape
previsores_reshaped = previsores.reshape(num_samples, timesteps, num_features)



In [162]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 84)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='pesos.h5', monitor='loss', save_best_only=True, verbose=1)

regressor.fit(previsores, preco_real, epochs = 100, batch_size = 32, callbacks=[es, rlr, mcp])

regressor.save_weights('regressor_ibov.h5')


Epoch 1/100
 8/10 [=======================>......] - ETA: 0s - loss: 0.0379 - mean_absolute_error: 0.1536
Epoch 1: loss improved from inf to 0.03706, saving model to pesos.h5
10/10 [==============================] - 7s 30ms/step - loss: 0.0371 - mean_absolute_error: 0.1528 - lr: 0.0010
Epoch 2/100
 9/10 [==========================>...] - ETA: 0s - loss: 0.0226 - mean_absolute_error: 0.1205
Epoch 2: loss improved from 0.03706 to 0.02346, saving model to pesos.h5
10/10 [==============================] - 1s 77ms/step - loss: 0.0235 - mean_absolute_error: 0.1218 - lr: 0.0010
Epoch 3/100
 9/10 [==========================>...] - ETA: 0s - loss: 0.0182 - mean_absolute_error: 0.1061
Epoch 3: loss improved from 0.02346 to 0.01816, saving model to pesos.h5
10/10 [==============================] - 0s 49ms/step - loss: 0.0182 - mean_absolute_error: 0.1056 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - ETA: 0s - loss: 0.0134 - mean_absolute_error: 0.0900
Epoch 4: loss improved fr